## Rating Table to Scorecard
**Authors**: Vinay Wunnava

In this tutorial, you are going to learn how to take a rating table from a `Generalized Additive 2 Model`, also known as `GA2M` and transform it into a scorecard.

**There are a few things that you need for this exercise:**

1. Your DataRobot API Key
2. A trained `Generalized Additive 2 Model` (on any project, with any dataset)

### Import Libraries

In [1]:
import csv
import pandas as pd
import numpy as np
import datarobot as dr

/Users/vinay.wunnava/anaconda3/lib/python3.7/site-packages/requests/__init__.py:91: RequestsDependencyWarning: urllib3 (1.26.2) or chardet (3.0.4) doesn't match a supported version!
  RequestsDependencyWarning)


### Connect to DataRobot

In [2]:
dr.Client(token = 'YOUR_API_TOKEN',
          endpoint = 'YOUR_ENDPOINT')

### Define Project and Model ID's.

The project and model ID's can be found in the url (when you use the UI to navigate). Make sure that the GA2M model does not have **any text features as input**.

In [3]:
pid = 'YOUR_PROJECT_ID'
mid = 'YOUR_MODEL_ID'

### Define Functions that download and transform the rating table to scorecard

In [4]:
def download_rating_table(pid, mid):
    """ Download the rating table corresponding to the pid and mid
    """
    project = dr.Project.get(pid)
    rating_tables = rating_tables = project.get_rating_tables()
    rating_table = [rt for rt in rating_tables if rt.model_id == mid][0]
    filepath = './my_rating_table_' + mid + '.csv'
    rating_table.download('./my_rating_table_' + mid + '.csv')
    return filepath

def csv_after_emptylines(filepath, bl_group_n=1, dtype=str):
    """ Read a .CSV into a Pandas DataFrame, but only after at least one blank line has been skipped.
    bl_group_n is the expected number of distinct blocks of blank lines (of any number of rows each) to skip before reading data.
    NB: E.g. pd.read_csv(filepath, skiprows=[0, 1, 2]) works if you know the number of rows to be skipped. Use this function if you have a variable / unknown number of filled rows (to be skipped / ignored) before the empty rows.
    """
    with open(filepath, newline='') as f:
        blank_lines = 0
        bl_groups = 0
        contents = []
        headers = None
        r = csv.reader(f)
        for i, l in enumerate(r):
            if bl_groups < bl_group_n:
                if not l:
                    blank_lines += 1
                    continue
                if blank_lines == 0:
                    continue
                bl_groups += 1
                blank_lines = 0
                headers = l
                continue
            contents.append(l)
        return pd.DataFrame(data=contents, columns=headers, dtype=dtype)

def csv_until_emptyline(filepath, dtype=str):
    """ Read a .CSV into a Pandas DataFrame until a blank line is found, then stop.
    """
    with open(filepath, newline='') as f:
        contents = []
        r = csv.reader(f)
        for i, l in enumerate(r):
            if not l:
                break
            if i == 0:
                headers = l
                continue
            contents.append(l)
        return pd.DataFrame(data=contents)

def extract_intercept(filepath):
    """ Read a .CSV into a Pandas DataFrame until a blank line is found, then stop.
        Extract intercept value and return it
    """ 
    df = csv_until_emptyline(filepath)
    df.rename(columns={df.columns[0]: "raw" }, inplace = True)
    df[['name','value']] = df['raw'].str.split(":",expand=True)
    intercept = pd.to_numeric(df.loc[df.name == 'Intercept','value'].values[0])
    return intercept

def invert_coefficients(intercept, rating_table):
    """ Inverting the sign of intercept and all the coefficients - this is to ensure that the high risk people are given low scores
        Mathematically, we are modelling log of odds and the riskier profiles have high probability
        When we negate the coefficients, it will mean the log of odds of non-risky profiles (- log(p/1-p) = log(1-p/p))
    """
    intercept = - intercept
    rating_table.loc[:,'Coefficient'] = - rating_table['Coefficient'].astype(float)
    return intercept, rating_table

def convert_rating_table_to_scores(intercept, rating_table, min_score=300, max_score=850):
    rating_table['Rel_Coefficient'] = rating_table['Coefficient']
    baseline = intercept
    min_sum_coef = 0
    max_sum_coef = 0
    for feat in rating_table['Feature Name'].unique():
        min_feat_coef = rating_table.loc[rating_table['Feature Name'] == feat]['Coefficient'].min()
        print('Minimum coefficient for feature ' + feat + ' ' + str(min_feat_coef))
        rating_table.loc[rating_table['Feature Name'] == feat,'Rel_Coefficient'] = rating_table['Coefficient'] - min_feat_coef
        baseline += min_feat_coef
        min_sum_coef = min_sum_coef + rating_table.loc[rating_table['Feature Name'] == feat]['Rel_Coefficient'].min()
        max_sum_coef = max_sum_coef + rating_table.loc[rating_table['Feature Name'] == feat]['Rel_Coefficient'].max()

    min_sum_coef = min_sum_coef + baseline
    max_sum_coef = max_sum_coef + baseline
    
    scaling_factor = (max_score - min_score) / (max_sum_coef - min_sum_coef)
    rating_table.loc[:,'Variable Score'] = rating_table['Rel_Coefficient'] * scaling_factor
    baseline_score = ((baseline - min_sum_coef) * scaling_factor) + min_score
    
    return baseline_score, rating_table.drop(columns=['Coefficient','Rel_Coefficient']), min_sum_coef, max_sum_coef

def get_scorecard(pid,mid, min_score=300, max_score=850):
    """ Download rating table for a particular pid and mid and return scorecard
    """
    filepath = download_rating_table(pid,mid)    
    rating_table_raw = csv_after_emptylines(filepath)
    intercept_raw = extract_intercept(filepath)
    intercept, rating_table = invert_coefficients(intercept_raw, rating_table_raw)
    intercept_score, scorecard, min_sum_coef, max_sum_coef = convert_rating_table_to_scores(intercept, rating_table, min_score, max_score)
    
    return intercept_score, scorecard, min_sum_coef, max_sum_coef

def get_score_from_prob(prob_default, min_score, max_score, min_sum_coef, max_sum_coef):
    """ Get score for probability of default and return score using the scorecard metrics - useful for threshold
    """
    prob_non_default = 1 - prob_default
    log_odds = np.log(prob_non_default / (1 - prob_non_default))
    scaling_factor = (max_score - min_score) / (max_sum_coef - min_sum_coef)
    score = ((log_odds - min_sum_coef) * scaling_factor) + min_score
    return score

### Create Scorecard and Intercept Score

In [5]:
intercept_score, scorecard, min_sum_coef, max_sum_coef = get_scorecard(pid,mid, min_score=300, max_score=850)

Minimum coefficient for feature annual_inc -0.7971001550635846
Minimum coefficient for feature dti -0.18100027283235265
Minimum coefficient for feature inq_last_6mths -0.3096886255271891
Minimum coefficient for feature revol_util_percent -0.6561420159264604
Minimum coefficient for feature ( inq_last_6mths & revol_util_percent ) -0.2953854956969044


In [6]:
print(intercept_score)
scorecard

300.0


,Feature Name,Feature Strength,Type,Transform1,Value1,Transform2,Value2,Weight,Variable Score
0,annual_inc,0.300775009108807,NUM,Binning,"(-inf, 13700.0]",,,57.0,0.000000
1,annual_inc,0.300775009108807,NUM,Binning,"(13700.0, 20200.0]",,,159.0,47.420770
2,annual_inc,0.300775009108807,NUM,Binning,"(20200.0, 21576.0]",,,32.0,49.394504
3,annual_inc,0.300775009108807,NUM,Binning,"(21576.0, 22489.0]",,,33.0,47.922292
4,annual_inc,0.300775009108807,NUM,Binning,"(22489.0, 24584.0]",,,125.0,30.520414
...,...,...,...,...,...,...,...,...,...
346,( inq_last_6mths & revol_util_percent ),0.033830128907363956,2W-INT,Binning,"(6, inf)",Binning,"(89, 91]",1.0,45.210374
347,( inq_last_6mths & revol_util_percent ),0.033830128907363956,2W-INT,Binning,"(6, inf)",Binning,"(91, 92]",0.0,43.279859
348,( inq_last_6mths & revol_util_percent ),0.033830128907363956,2W-INT,Binning,"(6, inf)",Binning,"(92, 95]",2.0,30.759709
349,( inq_last_6mths & revol_util_percent ),0.033830128907363956,2W-INT,Binning,"(6, inf)",Binning,"(95, inf)",0.0,31.991202


In [7]:
# Maximum probability of non-default based on rating table
max_prob_non_default = 1 / (1 + np.exp(- max_sum_coef))
print(f'Max probability of non-default: {max_prob_non_default}')
# Minimum probability of default would be 1 - max_prob_non_default
min_prob_default = 1 - max_prob_non_default
print(f'Min probability of default: {min_prob_default}')

Max probability of non-default: 0.9887734778925666
Min probability of default: 0.011226522107433357


In [8]:
# Minimum probability of non-default based on rating table
min_prob_non_default = 1 / (1 + np.exp(- min_sum_coef))
print(f'Min probability of non-default: {min_prob_non_default}')
# Maximum probability of default would be 1 - max_prob_non_default
max_prob_default = 1 - min_prob_non_default
print(f'Max probability of default: {max_prob_default}')

Min probability of non-default: 0.4345155432882876
Max probability of default: 0.5654844567117123


The scaling of the probabilities are <b>not</b> from \[0, 1\] to \[300, 850\] but from \[min_prob_non_default, max_prob_non_default\] to \[300, 850\]. The range of probabilities from the DR model correspond to the above values. The scaling applied here also correspond to the same range

In [9]:
# Convert threshold or any probabilty of default to score (the probabilty should lie between the min and max of the probability of default)
get_score_from_prob(0.1074, 300, 850, min_sum_coef, max_sum_coef) # f1 score opt - can be used as a threshold score

576.1843879406667

In [10]:
get_score_from_prob(0.0112, 300, 850, min_sum_coef, max_sum_coef) # Min prob default

850.2774648796886

In [11]:
get_score_from_prob(0.5654, 300, 850, min_sum_coef, max_sum_coef) # Max prob default

300.0398687633396